In [13]:
import pandas_datareader as pdr 
import pandas as pd 
from datetime import datetime, timedelta 
import yfinance as yf
import matplotlib.pyplot as plt 
import seaborn as sns 
import math 
import quantstats as qs

# pandas 설정 및 메타데이터 세팅 
pd.options.display.float_format = '{:.4f}'.format 
pd.set_option('display.max_columns', None)


pd.options.mode.chained_assignment = None

#date setting
start_day = datetime(2008,1,1) # 시작일 
end_day = datetime(2021,4,30) # 종료일 

'''# 공격자산 
SPY = pdr.get_data_yahoo('SPY', start_day - timedelta(days=365), end_day)['Adj Close']
VEA = pdr.get_data_yahoo('VEA', start_day - timedelta(days=365), end_day)['Adj Close']
EEM = pdr.get_data_yahoo('EEM', start_day - timedelta(days=365), end_day)['Adj Close']
AGG = pdr.get_data_yahoo('AGG', start_day - timedelta(days=365), end_day)['Adj Close']
        
# 수비자산
LQD = pdr.get_data_yahoo('LQD', start_day - timedelta(days=365), end_day)['Adj Close']
SHY = pdr.get_data_yahoo('SHY', start_day - timedelta(days=365), end_day)['Adj Close']
IEF = pdr.get_data_yahoo('IEF', start_day - timedelta(days=365), end_day)['Adj Close']
        '''

# RU : Risky Universe 
# CU : Cash Universe 
# BU : Benchmark Universe 
RU = ['SPY','VEA','EEM','AGG'] 
CU = ['LQD','SHY','IEF'] 
BU = ['^GSPC','^IXIC','^KS11','^KQ11'] # S&P500 지수, 나스닥 지수, 코스피 지수, 코스닥 지수

#tickers setting 
#tickers = ['SPY','VEA','VWO','AGG','SHY','IEF','LQD']



In [14]:
# 데이터 추출 함수 
def get_price_data(RU, CU, BU): 
    df_RCU = pd.DataFrame(columns=RU+CU) 
    df_BU = pd.DataFrame(columns=BU) 
    
    for ticker in RU + CU: 
        df_RCU[ticker] = pdr.get_data_yahoo(ticker, start_day - timedelta(days=365), end_day)['Adj Close'] 
        
    for ticker in BU: 
        df_BU[ticker] = pdr.get_data_yahoo(ticker, start_day - timedelta(days=365), end_day)['Adj Close'] 
        
    return df_RCU, df_BU


In [15]:
# 각 자산 군의 데이터 추출 

df_RCU, df_BU = get_price_data(RU, CU, BU)


In [16]:
df_RCU.head(5)

,SPY,VEA,EEM,AGG,LQD,SHY,IEF
Date,,,,,,,
2007-01-03,105.0776,nan,28.4739,63.8453,59.2579,65.3565,57.9124
2007-01-04,105.3006,nan,28.0807,63.9795,59.7032,65.4218,58.1225
2007-01-05,104.4607,nan,27.2597,63.9348,59.6643,65.3810,57.9334
2007-01-08,104.9438,nan,27.4575,63.9731,59.6476,65.3401,57.9755
2007-01-09,104.8545,nan,26.8442,63.9731,59.6587,65.3647,57.9615


In [17]:
df_RCU[RU+CU]

,SPY,VEA,EEM,AGG,LQD,SHY,IEF
Date,,,,,,,
2007-01-03,105.0776,nan,28.4739,63.8453,59.2579,65.3565,57.9124
2007-01-04,105.3006,nan,28.0807,63.9795,59.7032,65.4218,58.1225
2007-01-05,104.4607,nan,27.2597,63.9348,59.6643,65.3810,57.9334
2007-01-08,104.9438,nan,27.4575,63.9731,59.6476,65.3401,57.9755
2007-01-09,104.8545,nan,26.8442,63.9731,59.6587,65.3647,57.9615
...,...,...,...,...,...,...,...
2021-04-26,413.4663,49.9003,53.6639,113.1945,129.4029,86.0877,113.6585
2021-04-27,413.3773,49.7446,53.6345,112.9181,128.8228,86.0777,113.2113
2021-04-28,413.2584,49.8808,54.0367,112.9773,128.8425,86.0976,113.3604


In [6]:
# 모멘텀 지수 계산 함수 
def get_momentum(x): 
    temp_list = [0 for i in range(len(x.index))] 
    momentum = pd.Series(temp_list, index=x.index) 
    
    try: 
        before1 = df_RCU[x.name-timedelta(days=35):x.name-timedelta(days=30)].iloc[-1][RU+CU]
        before3 = df_RCU[x.name-timedelta(days=95):x.name-timedelta(days=90)].iloc[-1][RU+CU] 
        before6 = df_RCU[x.name-timedelta(days=185):x.name-timedelta(days=180)].iloc[-1][RU+CU] 
        before12 = df_RCU[x.name-timedelta(days=370):x.name-timedelta(days=365)].iloc[-1][RU+CU] 
        
        momentum = 12 * (x / before1 - 1) + 4 * (x / before3 - 1) + 2 * (x / before6 - 1) + (x / before12 - 1) 
        
    except Exception as e: 
        #print("Error : ", str(e)) 
        pass 
    
    return momentum

In [7]:
get_momentum(x)

NameError: name 'x' is not defined

In [8]:
# 각 자산별 모멘텀 지수 계산 

mom_col_list = [col+'_M' for col in df_RCU[RU+CU].columns] #column name

출처: https://lazyquant.tistory.com/entry/실전-백테스트-VAA-전략?category=951386 [게으른 퀀트]

SyntaxError: invalid syntax (<ipython-input-8-0e207b360b82>, line 5)

In [9]:
def VAAcalc(asset):
    df = yf.download(asset,start='1994-01-01')
    df['MA12'] =df['Adj Close'].rolling(window=365).mean()  #from 'AdjClose' column , 200days Moving Average ->MA200
    df['MA6'] =df['Adj Close'].rolling(window=180).mean()
    df['MA3'] =df['Adj Close'].rolling(window=120).mean() 
    df['MA1'] =df['Adj Close'].rolling(window=30).mean() 
    df['MA1'] =df['Adj Close'].rolling(window=30).mean() 
    df['M Score'] = (12*((df['Adj Close']/df['MA1'])-1)) + (4*((df['Adj Close']/df['MA3'])-1)) + (2*((df['Adj Close']/df['MA6'])-1)) + ((df['Adj Close']/df['MA12'])-1)
    df = df.dropna()
  #  df['price change'] = df['Adj Close'].pct_change() # daily return
  #  df['Upmove'] = df['price change'].apply(lambda x: x if x >0 else 0)
  #  df['Downmove'] = df['price change'].apply(lambda x: abs(x) if x < 0 else 0)
  #  df['avg Up'] = df['Upmove'].ewm(span=19).mean()
  #  df['avg Down'] = df['Downmove'].ewm(span=19).mean() 
  #  df['RS'] = df['avg Up'] / df['avg Down']
  #  df['RSI'] = df['RS'].apply(lambda x: 100 - (100/(x+1)))
  #  df.loc[(df['Adj Close'] > df['MA200']) & (df['RSI'] < 30), 'Buy'] = 'Yes'
  #  df.loc[(df['Adj Close'] < df['MA200']) | (df['RSI'] > 30), 'Buy'] = 'No'
    
    return df

In [10]:
s = VAAcalc(tickers[0])['M Score']



NameError: name 'tickers' is not defined

In [ ]:


for i in range(len(tickers)) : 
    globals()['frame_{}'.format(i)] = VAAcalc(tickers[i])['M Score'] # VAAcalc 는 series => frame_0, frame_1 ...
    
frame_M = pd.concat([frame_0, frame_1, frame_2, frame_3, frame_4, frame_5, frame_6], axis=1, keys = ['SPY_M','VEA_M','VWO_M','AGG_M','SHY_M','IEF_M','LQD_M'])
    
    # frame = frame []
    # Momentum_frame = 
    # matrixprofits.append(Profits)

In [ ]:
frame_M['Highest_Attact'] = frame_M[['SPY_M','VEA_M','VWO_M','AGG_M']].max(axis=1)
frame_M['Lowest_Attact'] = frame_M[['SPY_M','VEA_M','VWO_M','AGG_M']].min(axis=1)
frame_M['Highest_Defence'] = frame_M[['SHY_M','IEF_M','LQD_M']].max(axis=1)
frame_M['Lowest_Defence'] = frame_M[['SHY_M','IEF_M','LQD_M']].min(axis=1)


#if frame_M['Lowest_Attact'] >= 0 :
 #   frame_M['Choice'] = frame_M['Highest_Attact'] 
#else :
 #   frame_M['Choice'] = frame_M['Highest'] 
    

In [ ]:
frame_M

In [ ]:
# Keep Buying Method

#if frame_M['Lowest_Attact'] < 0 : # lowest among Attack 4 below 0
 #   frame_M['choice'] =  # lowest among Defence 3 below 0
        
        # put Highest value among Defence 3 'Choice' column
frame_M[['SHY_M','IEF_M','LQD_M']].idxmax(axis=1)